In [ ]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [5]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_torch  = torch.tensor(X_test,  dtype=torch.float32)
y_test_torch  = torch.tensor(y_test.to_numpy(),  dtype=torch.long)

train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset  = TensorDataset(X_test_torch,  y_test_torch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

In [6]:
n_qubits = 16
n_layers = 1

dev = qml.device('lightning.gpu', wires=n_qubits)

@qml.qnode(dev, interface="torch")
def qlayer(inputs, weights):
    qml.AngleEmbedding(inputs, wires=list(np.arange(n_qubits)), rotation='X')
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return tuple(qml.expval(qml.PauliZ(i)) for i in range(n_qubits))

In [7]:
X_train_sampled = []
y_train_sampled = []


for class_label in np.unique(y_train):
    class_indices = np.where(y_train == class_label)[0]
    sampled_indices = np.random.choice(class_indices, size=50, replace=False)

    X_train_sampled.extend(X_train[sampled_indices])
    y_train_sampled.extend(y_train.iloc[sampled_indices]) 

X_train_sampled = np.array(X_train_sampled)
y_train_sampled = np.array(y_train_sampled)

X_train_sampled_torch = torch.tensor(X_train_sampled, dtype=torch.float32)
y_train_sampled_torch = torch.tensor(y_train_sampled, dtype=torch.long)

train_sampled_dataset = TensorDataset(X_train_sampled_torch, y_train_sampled_torch)
train_sampled_loader = DataLoader(train_sampled_dataset, batch_size=32, shuffle=True)

In [ ]:
class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.scale = nn.Parameter(torch.tensor([2 * np.pi]))
        self.qnn_weights = torch.rand(n_layers, n_qubits, 3) * 1e-3
        self.output_layer = nn.Linear(n_qubits, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.tanh(self.fc4(x)) * self.scale
        batch_size = x.size(0)
        out = torch.empty((batch_size, n_qubits), dtype=torch.float32, device=x.device) 
        for batch_index in range(batch_size):
            expval_tensors = qlayer(x[batch_index], self.qnn_weights)
            expval_floats = [t.item() for t in expval_tensors]
            out[batch_index] = torch.tensor(expval_floats, device=x.device)
        x = self.output_layer(out)
        return x

In [ ]:
class ClassicalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.output_layer = nn.Linear(16, 10)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output_layer(x)
        return x

In [ ]:
model = ClassicalClassifier()

In [13]:
learning_rate = 1e-3
batch_size = 32
epochs = 30

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    model.to(torch_device)  
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(torch_device), y.to(torch_device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    pred = None
    X = None
    correct = 0
    model.to(torch_device)  
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(torch_device), y.to(torch_device)
            pred = model(X)
            predicted_label = torch.argmax(pred, dim=1)
            correct += torch.count_nonzero(predicted_label == y)
            test_loss += loss_fn(pred, y).item()

    print(f"Accuracy: {correct / (len(dataloader) * batch_size) * 100}")
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

    return test_loss

In [14]:
import time

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
start_time = time.time()

print(f"Before training sampled data\n--------------------------")
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop(train_sampled_loader, model, loss_fn, optimizer)
    test_loss = test_loop(test_loader, model, loss_fn)
    scheduler.step(test_loss)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop(test_loader, model, loss_fn)
print("Done!")

Before training sampled data
--------------------------
Epoch 1
--------------------------
loss: 2.322161 [   32/  500
Accuracy: 40.075626373291016
Avg loss: 2.075148 

Epoch 2
--------------------------
loss: 2.119331 [   32/  500
Accuracy: 54.48772430419922
Avg loss: 1.271622 

Epoch 3
--------------------------
loss: 1.209976 [   32/  500
Accuracy: 77.36872100830078
Avg loss: 0.777884 

Epoch 4
--------------------------
loss: 0.329156 [   32/  500
Accuracy: 81.67094421386719
Avg loss: 0.664084 

Epoch 5
--------------------------
loss: 0.185020 [   32/  500
Accuracy: 83.6900634765625
Avg loss: 0.606769 

Epoch 6
--------------------------
loss: 0.108664 [   32/  500
Accuracy: 83.6258544921875
Avg loss: 0.614081 

Epoch 7
--------------------------
loss: 0.060232 [   32/  500
Accuracy: 84.71746826171875
Avg loss: 0.594197 

Epoch 8
--------------------------
loss: 0.033301 [   32/  500
Accuracy: 84.28937530517578
Avg loss: 0.628388 

Epoch 9
--------------------------
loss: 0.039392

In [15]:
import time

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

start_time = time.time()

print(f"Before training\n--------------------------")
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loss = test_loop(test_loader, model, loss_fn)
    scheduler.step(test_loss)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop(test_loader, model, loss_fn)
print("Done!")

Before training
--------------------------
Epoch 1
--------------------------
loss: 0.971602 [   32/56000
loss: 0.083260 [ 3232/56000
loss: 0.226649 [ 6432/56000
loss: 0.138727 [ 9632/56000
loss: 0.213784 [12832/56000
loss: 0.102214 [16032/56000
loss: 0.083280 [19232/56000
loss: 0.145916 [22432/56000
loss: 0.256179 [25632/56000
loss: 0.090833 [28832/56000
loss: 0.172571 [32032/56000
loss: 0.105045 [35232/56000
loss: 0.658285 [38432/56000
loss: 0.290153 [41632/56000
loss: 0.268519 [44832/56000
loss: 0.174157 [48032/56000
loss: 0.447456 [51232/56000
loss: 0.214540 [54432/56000
Accuracy: 95.54793548583984
Avg loss: 0.154940 

Epoch 2
--------------------------
loss: 0.142550 [   32/56000
loss: 0.469733 [ 3232/56000
loss: 0.022658 [ 6432/56000
loss: 0.445050 [ 9632/56000
loss: 0.244947 [12832/56000
loss: 0.058959 [16032/56000
loss: 0.160407 [19232/56000
loss: 0.153737 [22432/56000
loss: 0.147507 [25632/56000
loss: 0.147264 [28832/56000
loss: 0.330473 [32032/56000
loss: 0.013570 [35232/5600